# Real experiment for Google Colab

The goal is to run the experiment with the real JRP instances. This notebook is prepared to be run in google colab.


## Conect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Installs

In [ ]:
# 1. RUN THIS
!git clone https://github.com/entropicalabs/openqaoa.git

In [ ]:
# 2. RUN THIS
!git clone https://github.com/AdrianoLusso/Tesis.git

In [4]:
# 3. RUN THIS
with open('Tesis/openqaoa_modifiedFiles/Makefile.txt', 'r') as file:
    replacement_content = file.read()

with open('openqaoa/Makefile', 'w') as file:
    file.write(replacement_content)

#verify a successful writing
#!cat openqaoa/Makefile

In [5]:
# 4. RUN THIS
with open('Tesis/openqaoa_modifiedFiles/requirements_core.txt', 'r') as file:
    replacement_content = file.read()

with open('openqaoa/src/openqaoa-core/requirements.txt', 'w') as file:
    file.write(replacement_content)

#verify a successful writing
#!cat openqaoa/src/openqaoa-core/requirements.txt

In [ ]:
!cd openqaoa && make local-install

## Imports

In [7]:
import sys
import os
from IPython.display import clear_output
from openqaoa.backends import create_device

from google.colab import files

sys.path.append('Tesis/functions')
from TestQAOASolver import TestQAOASolver

## Define configurations

In [8]:
# all the JRP initialization configurations for the JRPRandomGenerator
jrp_init_configurations = [
    {'num_agents':10 , 'num_vacnJobs':3 ,'penalty1':2.5,'penalty2':3, 'control_restrictions':False},
    {'num_agents':3 , 'num_vacnJobs':10 , 'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':6 , 'num_vacnJobs':5 ,'penalty1':2.5,'penalty2':3, 'control_restrictions':False},
    {'num_agents':5,'num_vacnJobs':6,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':7,'num_vacnJobs':4,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':4,'num_vacnJobs':7,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':8,'num_vacnJobs':3,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':3,'num_vacnJobs':8,'penalty1':2.5,'penalty2':3,'control_restrictions':False}
]

In [9]:
# uncomment ONLY if you want to do some test within simpler instances
'''
jrp_init_configurations = [
    {'num_agents':2 , 'num_vacnJobs':3 ,'penalty1':2.5,'penalty2':3, 'control_restrictions':False},
    {'num_agents':3 , 'num_vacnJobs':2 , 'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':2 , 'num_vacnJobs':4 ,'penalty1':2.5,'penalty2':3, 'control_restrictions':False},
    {'num_agents':4,'num_vacnJobs':2,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':3,'num_vacnJobs':3,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':3,'num_vacnJobs':3,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':5,'num_vacnJobs':2,'penalty1':2.5,'penalty2':3,'control_restrictions':False},
    {'num_agents':2,'num_vacnJobs':5,'penalty1':2.5,'penalty2':3,'control_restrictions':False}
]
'''


In [10]:
circuit_configurations =[
    {
    'p': 4,
    'param_type':'standard',
    'init_type':'ramp',
    'mixer_hamiltonian':'x'
    },
    {
    'p': 5,
    'param_type':'standard',
    'init_type':'ramp',
    'mixer_hamiltonian':'x'
    },
    {
    'p': 6,
    'param_type':'standard',
    'init_type':'ramp',
    'mixer_hamiltonian':'x'
    },
    {
    'p': 7,
    'param_type':'standard',
    'init_type':'ramp',
    'mixer_hamiltonian':'x'
    }
]

In [11]:
optimization_backend_configuration ={
    'n_shots':10000
    #'qiskit_simulation_method':'statevector'
    }

evaluation_backend_configuration ={
    'n_shots':20
    #'qiskit_simulation_method':'matrix_product_state'
    }

In [12]:
optimizer_configuration={
    'method' : 'Powell',
    'maxfev':10000,
    'tol':0.01,
    'optimization_progress':True,
    'cost_progress':True,
    'parameter_log':True
    }

## Sampling

In [ ]:
# matrix where an element [i,j] is the size of the sample for:
#   - JRP init configuration 'i'
#   - circuit configuration 'j'
ns_samples = [
      [3,3,2,2]
      ,[3,3,2,2]
       ,[6,6,4,4]
       ,[6,6,4,4]
       ,[3,3,2,2]
       ,[3,3,2,2]
       ,[3,3,2,2]
       ,[3,3,2,2]
    ]
solver = TestQAOASolver()

for jrp_init_itr,jrp_init_conf in enumerate(jrp_init_configurations):
  for circuit_itr,circuit_conf in enumerate(circuit_configurations):
    print('JRP CONFIGURATION ',jrp_init_itr,' , CIRCUIT CONFIGURATION ',circuit_itr)

    name = '_jrp'+str(jrp_init_itr)+'_circuit'+str(circuit_itr)
    n_samples = ns_samples[jrp_init_itr][circuit_itr]

    solver.sample_workflows(
        name,
        n_samples,
        jrp_init_conf,
        circuit_conf,
        optimizer_configuration,
        optimization_backend_configuration,
        evaluation_backend_configuration
    )

    file_name = 'conf'+name+'.json'
    !cp {file_name} /content/drive/MyDrive/results_experiment_colab
    clear_output(wait=True)